### Let's see what this looks like 

In [78]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt

#### I'll be testing functions on this abbreviated chunk of one file

In [186]:
test_df = pd.read_csv('data/2017-fordgobike-tripdata.csv').head(3000)

In [187]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 13 columns):
duration_sec               3000 non-null int64
start_time                 3000 non-null object
end_time                   3000 non-null object
start_station_id           3000 non-null int64
start_station_name         3000 non-null object
start_station_latitude     3000 non-null float64
start_station_longitude    3000 non-null float64
end_station_id             3000 non-null int64
end_station_name           3000 non-null object
end_station_latitude       3000 non-null float64
end_station_longitude      3000 non-null float64
bike_id                    3000 non-null int64
user_type                  3000 non-null object
dtypes: float64(4), int64(4), object(5)
memory usage: 304.8+ KB


In [55]:
folder = 'data/'
files = [folder+file for file in os.listdir(folder) if not file.startswith('.')] 

In [49]:
target_columns = ['start_time', 'end_time', 
                             'start_station_latitude', 'start_station_longitude',
                             'end_station_latitude', 'end_station_longitude'] #most important for now

### THERE IS JUST OVER 1GB OF DATA ACROSS 27 CSV'S AND WE WON'T BE ABLE TO CONCAT THIS ALL INTO 1 DATAFRAME
#### I'LL WRITE FUNCTIONS THAT WILL BE CALLED ON INDIVIDUAL DFs
#### >FIRST WE'LL WANT TO CHECK TO SEE IF OUR THE MOST IMPORTANT COLUMNS HAVE NULL VALUES TO BE CLEANED

In [258]:
class BikeData():
    def __init__(self, files):
        self.files = files
        
    def do_on_all(self, func):  
        for file in self.files:
            current_df = pd.read_csv(file)
            func(current_df, file)
                
    def check_problem_columns(self, current_df, current_file, columns_that_cant_be_null=target_columns):        
        problem_columns = []
        for col in columns_that_cant_be_null:
            problem_rows = current_df[current_df[col].isnull()]
            if problem_rows.size > 0:
                problem_columns.append(col)
            if len(problem_columns) > 0:
                print (current_file)
                print (problem_problem_columns)
        return
    
    def rentals_per_month(self, current_df=None, current_file=None):
        rentals = current_df['start_time'].size
        print ("{0}  :  {1}".format(current_file[5:11], rentals))
        return
    
    def weekend_averages(self, current_df, current_file):
        time = current_df['start_time']
        
        current_df['day'] = time.apply(lambda date: date.split('-')[2][:2]) #the original string is formatted y-m-d t:i:m:e
        
        current_df['weekends'] = (time.appy(lambda date: pd.to_datetime(date))
                                      .apply(lambda date: date.weekday() in [5,6])) #dt.weekday returns a list value, 5 & 6 are Sat & Sun
        
        wk_group = current_df.groupby('weekend')['day']
        number_of_days = int(current_df['day'].max()) #returns 28, 30 or 31 depending on length of month
        
        number_of_weekends = wk_group.nunique()[True]
        number_of_weekdays = number_of_days - number_of_weekends
        
        weekend_ratio = number_of_weekends/number_of_days
        weekday_ratio = number_of_weekdays/number_of_days
        
        total_trips = current_df['day'].size
        
        expected_weekend_trips = int(total_trips*weekend_ratio)
        expected_weekday_trips = int(total_trips*weekday_ratio)
        
        actual_weekend_trips = wk_group.size()[True]
        actual_weekday_trips = wk_group.size()[False]

        print (current_file[5:11])
        print ("Total Weekend Trips: {0} | ")
                

In [246]:
all_data = BikeData(files)

In [234]:
all_data.do_on_all([check_problem_columns])

##### everything seems to be in working order -- GREAT

In [81]:
print (test_df['start_time'][0])
print (type(test_df['start_time'][0]))

2019-02-28 17:32:10.1450
<class 'str'>


In [188]:
test_df['weekend'] = test_df['start_time'].apply(lambda date: pd.to_datetime(date)).apply(lambda date: date.weekday() in [5,6])

In [189]:
test_df['day'] = test_df['start_time'].apply(lambda date: date.split('-')[2][:2])

In [215]:
test_df.groupby('weekend')['day'].nunique()

weekend
False    1
True     2
Name: day, dtype: int64

In [223]:
test_df.groupby('weekend')['day'].size()[True]

2854

In [220]:
test_df['day'].size

3000

In [177]:
test_df.tail()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,weekend
995,456,2017-12-31 12:44:59.8500,2017-12-31 12:52:35.9240,249,Russell St at College Ave,37.858473,-122.253253,249,Russell St at College Ave,37.858473,-122.253253,309,Customer,True
996,651,2017-12-31 12:41:40.1680,2017-12-31 12:52:31.7810,74,Laguna St at Hayes St,37.776435,-122.426244,123,Folsom St at 19th St,37.760594,-122.414817,1665,Subscriber,True
997,224,2017-12-31 12:48:17.9220,2017-12-31 12:52:02.3550,93,4th St at Mission Bay Blvd S,37.770407,-122.391198,81,Berry St at 4th St,37.775880,-122.393170,3010,Subscriber,True
998,2804,2017-12-31 12:04:57.8690,2017-12-31 12:51:42.3310,134,Valencia St at 24th St,37.752428,-122.420628,15,San Francisco Ferry Building (Harry Bridges Pl...,37.795392,-122.394203,1658,Customer,True
999,2784,2017-12-31 12:05:09.8680,2017-12-31 12:51:34.7460,134,Valencia St at 24th St,37.752428,-122.420628,15,San Francisco Ferry Building (Harry Bridges Pl...,37.795392,-122.394203,3006,Customer,True


In [259]:
all_data.do_on_all(rentals_per_month)

TypeError: rentals_per_month() missing 1 required positional argument: 'current_file'

In [244]:
rentals_permonth()

NameError: name 'rentals_permonth' is not defined